### Let's see if the API key works

In [1]:
import langchain
import os
from model import init_llm
from dotenv import load_dotenv
from prompts import get_blurb_prompt
import random
import time 
import json

verbose = False
load_dotenv()
file_path = 'dataset.jsonl'

In [2]:
# Utility functions

# Function to randomly select subsets of varying sizes
def select_random_subsets(input_list, num_subsets):
    subsets = []
    list_length = len(input_list)
    
    for _ in range(num_subsets):
        subset_size = random.randint(1, list_length)  # Random subset size
        subset = random.sample(input_list, subset_size)
        subsets.append(subset)
    
    return subsets

In [3]:
# initialize the language model
# llm = init_llm()

# prompt = get_blurb_prompt()

# gen_query_chain = langchain.LLMChain(
#         llm=llm,
#         prompt=prompt,
#         verbose=verbose
#     )

In [4]:
# Pre-defined feature sets

features = ['price', 'phone_number', 'color', 'dimension']
positive = ['color', 'dimension']
negative = ["price", "phone_number"]

colors = ["red", "blue", "green", "yellow", "black", "white", "purple", "orange", "pink", "brown"]
phone_numbers = ["123-456-7890", "234-567-8901", "345-678-9012", "456-789-0123", "567-890-1234", "678-901-2345", "789-012-3456", "890-123-4567", "901-234-5678", "012-345-6789"]
prices = ["$100", "$200", "$300", "$400", "$500", "$600", "$700", "$800", "$900", "$1000"]
dimensions = ["1x1", "2x2", "3x3", "4x4", "5x5", "6x6", "7x7", "8x8", "9x9", "10x10"]

data = {'color': colors, 'phone_number': phone_numbers, 'price': prices, 'dimension': dimensions}


In [6]:
random_subsets = select_random_subsets(features, 5)

ground_truths = []

for idx, feature_set in enumerate(random_subsets):

    ground_truth = []
    vals = []

    for feature in feature_set:
        if feature == "dimension":
            dim = random.choice(data[feature])
            ground_truth.append(dim)
            vals.append(dim)
        
        if feature == "color":
            col = random.choice(data[feature])
            ground_truth.append(col)
            vals.append(col)

        if feature == "price":
            pri = random.choice(data[feature])
            vals.append(pri)

        if feature == "phone_number":
            pho = random.choice(data[feature])
            vals.append(pho)

    #output = gen_query_chain.predict(features=feature_set, values=vals)
    output = 'Hello there my boy'

    temp = {"Query": output, "Feature Set": feature_set, "Values": vals, "Ground Truth": ground_truth}

    with open(file_path, "a") as f:
        json.dump(temp, f)  # Write JSON object
        f.write('\n')       # Add a newline to separate objects

In [7]:
with open(file_path, "r") as f:
    for line in f:
        # Parse JSON object from the line
        read = json.loads(line)
        
        # Process the data (for example, print it)
        print(read)


{'Query': "Introducing the sleek and stylish Widget-X, available now in a stunning white hue that exudes sophistication. With its cutting-edge design and top-notch features, this innovative gadget is the perfect companion for your everyday adventures. Dial up your connectivity with ease using the included phone_number: 345-678-9012. And the best part? This premium device comes at an unbeatable price of just $100, making it a must-have for tech enthusiasts everywhere. Don't miss out on the opportunity to experience the future of communication in the palm of your hand. Get your Widget-X today!", 'Feature Set': ['phone_number', 'color', 'price'], 'Values': ['345-678-9012', 'white', '$100'], 'Ground Truth': ['white']}
{'Query': "Discover the wonder of infinite possibilities with our latest creation: the Dimension-X cube, measuring an astounding 9x9 in size. Immerse yourself in a realm where imagination knows no bounds as you explore the depths of this captivating dimension. Whether you're 